# TASHO - A model predictive control toolchain for constraint-based task specification of robot motions

#### Presented by: Alejandro Astudillo and Ajay Sathya

Tasho stands for “**TA**sk **S**pecification with receding **HO**rizon control”



| <img src="files/img/overview.svg" width="800"> | 
|:--:| 
| Fig. 1. Tasho's overview |

An implementation of a contour-following MPC using **Tasho**

  
First, we need to import the relevant modules from Tasho and other dependencies (casadi, numpy)

In [ ]:
import os
os.chdir(str(os.getcwd())+"/../../")

import sys
from tasho import task_prototype_rockit as tp
from tasho import input_resolution
from tasho import robot as rob
from tasho import environment as env
import casadi as cs
from rockit import MultipleShooting, Ocp
import numpy as np

## Robot model

Import the robot object from the robot's repository

In [ ]:
# robot_choice = "yumi"
robot_choice = "kinova"
# robot_choice = "iiwa7"
# robot_choice = "atlas"
# robot_choice = "ur10"

robot = rob.Robot(robot_choice, analytical_derivatives=True)

The robot object includes functions for forward dynamics, inverse dynamics, forward kinematics, and their derivatives, beside variables for joint limits and barycentric parameters.

#### Forward dynamics:

$
\ddot{\mathbf{q}} = M^{-1}(\mathbf{q})\left(\tau - C(\mathbf{q}, \dot{\mathbf{q}})\dot{\mathbf{q}} - G(\mathbf{q})\right)
$

In [ ]:
FD = robot.fd
print(FD)

In [ ]:
J_FD = robot.J_fd
print(J_FD)

#### Inverse dynamics:
$
\tau = M(\mathbf{q}) + C(\mathbf{q}, \dot{\mathbf{q}})\dot{\mathbf{q}} + G(\mathbf{q})
$

In [ ]:
ID = robot.id
print(ID)

In [ ]:
J_ID = robot.J_id
print(J_ID)

#### Forward kinematics:

In [ ]:
FK = robot.fk
print(FK)

#### Upper and lower bound on joint position:

In [ ]:
print("Upper bound on q: ",robot.joint_ub)

In [ ]:
print("Lower bound on q: ",robot.joint_lb)

#### Upper and lower bound on joint velocity:

In [ ]:
print("Upper bound on q_dot: ",robot.joint_vel_ub)

In [ ]:
print("Lower bound on q_dot: ",robot.joint_vel_lb)

#### Upper and lower bound on joint torque:

In [ ]:
print("Upper bound on tau: ",robot.joint_torque_ub)

In [ ]:
print("Lower bound on tau: ",robot.joint_torque_lb)

#### Upper and lower bound on joint acceleration:

In [ ]:
print("Upper bound on q_ddot: ",robot.joint_acc_ub)

In [ ]:
print("Lower bound on q_ddot: ",robot.joint_acc_lb)

#### Update robot's parameters if needed

In [ ]:
ocp_control = "acceleration_resolved"
# ocp_control = "torque_resolved"

if ocp_control == "acceleration_resolved":
    max_joint_acc = 240 * cs.pi / 180
    robot.set_joint_acceleration_limits(lb=[-max_joint_acc]*robot.ndof, ub=[max_joint_acc]*robot.ndof)
    

In [ ]:
print("Upper bound on q_ddot: ",robot.joint_acc_ub)

In [ ]:
print("Lower bound on q_ddot: ",robot.joint_acc_lb)

## Task spacification

In [ ]:
# Define initial conditions of the robot
if robot_choice == "yumi":
    left_arm_q_init = [-1.35, -3.72e-01, 2.18, 0.78e-01, 2.08, -9.76e-01, -1.71, 1.65e-03, 1.65e-03]
    # Right arm in home configuration
    right_arm_q_init = [0, -2.26, -2.35, 0.52, 0.025, 0.749, 0, 0, 0]

    q_init = np.array(left_arm_q_init + right_arm_q_init).T
elif robot_choice == "kinova":
    q_init = [0, -0.523598, 0, 2.51799, 0, -0.523598, -1.5708]
q_dot_init = [0] * robot.ndof

Select prediction horizon and sample time for the MPC execution

In [ ]:
horizon_size = 20
t_mpc = 0.1

Initialize the task context object

In [ ]:
tc = tp.task_context(horizon_size * t_mpc, horizon_steps = horizon_size)

Define the input type of the robot (torque or acceleration)

In [ ]:
q, q_dot, q_ddot, q0, q_dot0 = input_resolution.acceleration_resolved(tc, robot)

Add object of interest for the robot (in this case a cube + 0.05 m in $z$-direction)

In [ ]:
cube_pos = tc.create_expression("cube_pos", "parameter", (3, 1))
T_goal = cs.vertcat(
    cs.hcat([0, 1, 0, cube_pos[0]]),
    cs.hcat([1, 0, 0, cube_pos[1]]),
    cs.hcat([0, 0, -1, cube_pos[2]+0.05]),
    cs.hcat([0, 0, 0, 1]),
)

Define constraints at the end of the horizon (final ee position and final joint velocity)

In [ ]:
T_ee = robot.fk(q)[7]

final_pos = {
    "hard": False,
    "type": "Frame",
    "expression": T_ee,
    "reference": T_goal,
    "rot_gain": 10,
    "trans_gain": 10,
    "norm": "L1",
}
final_vel = {"hard": True, "expression": q_dot, "reference": 0}
final_constraints = {"final_constraints": [final_pos, final_vel]}
tc.add_task_constraint(final_constraints)

Add penality terms on joint velocity and acceleration for regulatization

In [ ]:
tc.add_regularization(
    expression=q_dot, weight=1e-3, norm="L2", variable_type="state", reference=0
)
tc.add_regularization(
    expression=q_ddot, weight=1e-3, norm="L2", variable_type="control", reference=0
)

Set solver and discretization options

In [ ]:
tc.set_ocp_solver(
    "ipopt",
    {
        "ipopt": {
            "print_level": 0,
            "tol": 1e-3,
        }
    },
)

In [ ]:
disc_settings = {
    "discretization method": "multiple shooting",
    "horizon size": horizon_size,
    "order": 1,
    "integration": "rk",
}
tc.set_discretization_settings(disc_settings)

Set parameter values

In [ ]:
tc.ocp.set_value(cube_pos, [0.5, 0, 0.25])
tc.ocp.set_value(q0, q_init)
tc.ocp.set_value(q_dot0, q_dot_init)

Solve the OCP that describes the task

In [ ]:
sol = tc.solve_ocp()


## MPC Simulation

Create world simulator based on pybullet

In [ ]:
from tasho import world_simulator
import pybullet as p

obj = world_simulator.world_simulator()

Add robot to the world environment

In [ ]:
position = [0.0, 0.0, 0.0]
orientation = [0.0, 0.0, 0.0, 1.0]
robotID = obj.add_robot(position, orientation, robot_choice)

Define world environment

In [ ]:
environment = env.Environment()

Add cube to the environment

In [ ]:
cube1 = env.Cube(length = 1, position = [0.5, -0.2, 0.35], orientation = [0.0, 0.0, 0.0, 1.0], urdf = "/models/objects/cube_small.urdf")
environment.add_object(cube1, "cube")

Add table to the environment

In [ ]:
table1 = env.Box(height = 0.3, position = [0.5, 0, 0], orientation = [0.0, 0.0, 0.7071080798594737, 0.7071054825112364], urdf = "/models/objects/table.urdf")
environment.add_object(table1, "table1")

Add environment to world simulator

In [ ]:
environment.set_in_world_simulator(obj)

Update initial velocity of the cube

In [ ]:
cubeID = environment.get_object_ID("cube")
p.resetBaseVelocity(cubeID, linearVelocity=[0, 0.7, 0])

Determine number of samples that the simulation should be executed

In [ ]:
no_samples = int(t_mpc / obj.physics_ts)

Correspondence between joint numbers in bullet and OCP

In [ ]:
if robot_choice == "yumi":
    joint_indices = [11, 12, 13, 14, 15, 16, 17, 18, 19, 1, 2, 3, 4, 5, 6, 7, 8, 9]
elif robot_choice == "kinova":
    joint_indices = [0, 1, 2, 3, 4, 5, 6]

Begin the visualization by applying the initial control signal

In [ ]:
ts, q_sol = sol.sample(q, grid="control")
ts, q_dot_sol = sol.sample(q_dot, grid="control")
obj.resetJointState(robotID, joint_indices, q_init)
obj.setController(
    robotID, "velocity", joint_indices, targetVelocities=q_dot_sol[0]
)

Execute the MPC loop

In [ ]:
for i in range(horizon_size * 100):
    print("----------- MPC execution -----------")

    # Predict the position of the target object (cube)
    lin_vel, ang_vel = p.getBaseVelocity(cubeID)
    lin_vel = cs.DM(lin_vel)
    lin_pos, _ = p.getBasePositionAndOrientation(cubeID)
    lin_pos = cs.DM(lin_pos)
    time_to_stop = cs.norm_1(lin_vel) / 0.5
    predicted_pos = (
        cs.DM(lin_pos)
        + cs.DM(lin_vel) * time_to_stop
        - 0.5 * 0.5 * lin_vel / (cs.norm_1(lin_vel) + 1e-3) * time_to_stop ** 2
    )
    predicted_pos[2] += 0.03  # cube height
    print("Predicted position of cube", predicted_pos)

    # Set parameter values
    tc.ocp.set_value(q0, q_sol[1])
    tc.ocp.set_value(q_dot0, q_dot_sol[1])
    tc.ocp.set_value(cube_pos, predicted_pos)

    # Solve the ocp
    sol = tc.solve_ocp()

    # Sample the solution for the next MPC execution
    ts, q_sol = sol.sample(q, grid="control")
    ts, q_dot_sol = sol.sample(q_dot, grid="control")

    tc.ocp.set_initial(q, q_sol.T)
    tc.ocp.set_initial(q_dot, q_dot_sol.T)

    # Set control signal to the simulated robot
    obj.setController(
        robotID, "velocity", joint_indices, targetVelocities=q_dot_sol[0]
    )

    # Simulate
    obj.run_simulation(no_samples)

    # Termination criteria
    T_ee_sol = robot.fk(q_sol[0])[7]
    pos_ee_sol = T_ee_sol[:3, 3]+np.array([0,0,0.05])
    dist_to_cube_sq = cs.sumsqr(pos_ee_sol - predicted_pos)
    if dist_to_cube_sq <= 2e-2 ** 2:
        break

obj.run_simulation(100)

obj.end_simulation()